### 번역자: 고려대학교 수학과(12) 최시현
[AiKorea 번역 프로젝트](https://github.com/aikorea/cs231n)를 이어받아서 진행하였습니다. 기존 AiKorea에서 이미 번역되어있는 CourseNote의 경우 유사한 부분이 많습니다. <br /> 
(깨진 삽화를 추가하거나 개인적으로 번역이 매끄럽지 못하다 생각하는 부분은 수정하였습니다.) <br />
문제가 될시에는 자진 삭제하겠습니다. 오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

## 소개(Introduction)

이전 섹션에서 이미지 분류를 할 때에 있어 두 가지의 핵심요소를 소개하였다.
- Raw 이미지의 픽셀들을 클래스 스코어에 매핑시켜주는 Parameterized된 스코어 함수
- 특정 파라미터들의 집합의 quality를 측정하는 손실함수. 이는 계산된 스코어가 학습 데이터의 ground truth 라벨에 대하여 얼마나 잘 agreed with 하는지에 기초한다. <br />
(e.g: Softmax/SVM)

구체적으로 말하자면, 다음과 같은 형식을 가진 선형함수 $f\left( { x }_{ i },W \right) =W{ x }_{ i }$를 스코어 함수로 쓸 때, 앞에서 다룬 바와 같이 SVM은 다음과 같은 수식으로 표현할 수 있다.

<img src="./Images/1.png" width=600 />

${x}_{i}$에 대한 예측값이 실제 값(레이블, Labels) ${y}_{i}$와 같아지도록 세팅된 파라미터들 $W$는 손실값 L 또한 매우 낮게 나온다는 것을 알게되었다. <br />
이제 세번째이자 마지막 핵심요소인 최적화에 대해서 알아보자. 최적화는 손실함수를 최소화시키는 파라미터들의 집합 $W$를 찾는 과정을 의미한다.

## 손실함수의 시각화

이 강의에서 다루는 손실함수들은 대체로 고차원 공간에서 정의된다. <br /> 
(예를 들어 CIFAR-10의 선형분류기의 경우 파라미터 행렬은 크기가 [10x3073]이고총 30,703개의 파라미터가 있다.) <br /> 
따라서 시각화하기가 어렵다. 하지만 고차원 공간을 1차원 직선이나 2차원 평면으로 잘라서 보면 어느정도 직관을 얻을 수 있다. <br />
예를 들어, 무작위로 가중치 행렬 $W$를 하나 생성한다고 가정해보자. (이는 사실 고차원 공간의 한 점인 셈이다) 이제 이 점을 직선 하나를 따라 이동시키면서 손실함수를 기록해보자 <br />
즉, 무작위로 뽑은 방향 ${W}_{1}$을 잡고, 이 방향을 따라 가면서 손실함수를 계산하는데, 즉, $L(W+a{ W }_{ 1 })$에 여러 개의 a 값을 넣어 계산해 보는 것이다. <br />
이 과정을 통해 우리는 a값을 x축, 손실 함수 값을 y축에 놓고 간단한 그래프를 그릴 수 있다. 또한 이 비슷한 것을 2차원으로도 할 수 있다. <br />
여러 a, b값에 따라 $L(W+a{ W }_{ 1 }+a{W}_{2})$을 계산하고 a, b는 각각 x축과 y축에, 손실 함수 값을 색을 이용해 그리면 된다.
___

<img src="./Images/2.png" width=800 />

정규화(regularization) 없는 멀티 클래스 SVM의 손실함수의 지형을 CIFAR-10 데이터의 1개 예시(왼쪽, 가운데)와 여러 개의 예시(오른쪽)에 적용시켜 그려본 그림들 <br />
왼쪽: 여러 a값에 따른 1차원 손실 곡선 <br />
가운데, 오른쪽: 2차원 손실 평면, 파란색은 낮은 손실을 뜻하고, 빨간색은 높은 손실을 뜻한다. 손실 함수가 piecewise-linear structure인 것이 특징이다. <br />
특히, 오른쪽 그림은 여러 예시를 통해 구한 손실들에 대하여 평균을 낸 것인데, bowl 모양인 것이 특징이다. 이는 가운데 처럼 piece-wise linear bowls 여러 개를 평균낸 모양이다.
___

손실함수의 pieewise-linear structure구조는 수식을 통해 설명할 수 있다. 예시가 하나인 경우에 다음과 같이 쓸 수 있다. 

<img src="./Images/3.png" width=400 />

수식에서 볼 수 있듯이, 각 예시의 손실값은 ($max(0,-)$함수로 0에서 threshold를 가지고 있는) $W$의 선형함수들의 합으로 표현된다. <br />
$W$의 각 행(즉, ${w}_{j}$) 앞에는 잘못된 클래스와 대응되는 경우 positive sign이 붙고, 옳은 클래스와 대응되는 경우 negative sign이 붙는다. <br />
예를 들어, 3개의 1차원 점들과 3개의 클래스가 있다고 해보자. 정규화(regulation)가 없는 총 SVM 손실은 다음과 같다.

<img src="./Images/4.png" width=600 />

이 예시들은 1차원이기 때문에, 데이터 ${x}_{i}$와 파라미터 ${w}_{i}$는 숫자이다. <br />
예를 들어, ${w}_{0}$를 살펴보면 위에서 어떤 항들은 ${w}_{0}$에 대한 선형함수이고 각각은 0에서 꺽인다. 이를 다음과 같이 시각화 할 수 있다. 
___

<img src="./Images/5.png" width=700 />

손실을 1차원으로 표현한 그림 <br />
x축은 단일 가중치이고, y축은 손실이다. 손실은 여러 항들의 합인데, 각각은 특정 가중치에 대하여 독립적이거나, 0을 threshold로 하는 선형함수 이다. <br />
full SVM data 손실은 이 모양의 30,730 차원 버전이다.
___

아마도 bowl 모양을 보고 SVM 손실함수가 일종의 [볼록 함수](https://en.wikipedia.org/wiki/Convex_function)라고 생각했을 것이다. <br />
이런 형태의 함수를 효율적으로 최소화하는 문제에 대한 엄청난 양의 연구 성과들이 있다.(Stanford class on the topic([convex optimization](http://stanford.edu/~boyd/cvxbook/))) <br />
하지만 이 스코어 함수 $f$를 뉴럴 네트워크로 확장시키면, 목적 함수(objective functions)는 더 이상 볼록함수가 아니게 되고, <br /> 
위와 같은 시각화를 해봐도 bowl 모양 대신 울퉁 불퉁하고 복잡한 모양이 보일 것이다.

#### 미분이 불가능한 손실함수

기술적인 설명을 덧붙이자면, $max(0,-)$함수 때문에 손실함수에 꺽임이 생기는데, 이 때문에 손실함수의 미분이 불가능해 진다. <br />
왜냐하면, 그 꺽이는 부분에서 미분 혹은 그라디언트가 존재하지 않기 때문이다. 하지만 서브 그라디언트([subgradient](https://en.wikipedia.org/wiki/Subderivative))가 존재하고, 대체로 이를 그라디언트 대신 사용할 것이다. <br />
앞으로 이 강의에서는 그라디언트와 서브 그라디언트를 구분하지 않고 쓸 것이다.

## 최적화

정리하면, 손실함수는 어떤 특정 가중치들의 집합 $W$의 quality를 측정한다. 최적화의 목적은 이 손실함수를 최소화시키는 $W$를 찾아내는 것이다. <br />
다음 단락부터 손실함수를 최적화하는 방법에 대해서 살펴볼 것이다. <br />
우리의 궁극적인 목적은 뉴럴 네트워크를 최적화시키는 것이고, 여기에는 볼록 함수 최적화를 위해 고안된 방법들은 통하지 않는다.

#### 전략1: 첫번째, 매우 나쁜 방법: 무작위 탐색

주어진 파라미터 $W$가 얼마나 좋은지를 측정하는 것은 매우 간단하기 때문에, 처음 떠오르는 생각은 단순히 무작위로 파라미터들을 골라서 넣어보고, <br /> 
넣어 본 값들 중 제일 좋은 값을 기록하는 것이다. 그 과정은 다음과 같다.

In [ ]:
# assume X_train is the data where each column is an example (e.g. 3073 x 50,000)
# assume Y_train are the labels (e.g. 1D array of 50,000)
# assume the function L evaluates the loss function

bestloss = float("inf") # Python assigns the highest possible float value
for num in xrange(1000):
  W = np.random.randn(10, 3073) * 0.0001 # generate random parameters
  loss = L(X_train, Y_train, W) # get the loss over the entire training set
  if loss < bestloss: # keep track of the best solution
    bestloss = loss
    bestW = W
  print 'in attempt %d the loss was %f, best %f' % (num, loss, bestloss)

# prints:
# in attempt 0 the loss was 9.401632, best 9.401632
# in attempt 1 the loss was 8.959668, best 8.959668
# in attempt 2 the loss was 9.044034, best 8.959668
# in attempt 3 the loss was 9.278948, best 8.959668
# in attempt 4 the loss was 8.857370, best 8.857370
# in attempt 5 the loss was 8.943151, best 8.857370
# in attempt 6 the loss was 8.605604, best 8.605604
# ... (trunctated: continues for 1000 lines)

위의 코드에서, 여러 개의 무작위 가중치 벡터 $W$를 넣어봤고, 그 중 몇몇은 다른 것들보다 좋았다. <br /> 
그래서 그 중 제일 좋은 가중치 $W$를 테스트 데이터에 대하여 시도해 보면된다.

In [ ]:
# Assume X_test is [3073 x 10000], Y_test [10000 x 1]
scores = Wbest.dot(Xte_cols) # 10 x 10000, the class scores for all test examples
# find the index with max score in each column (the predicted class)
Yte_predict = np.argmax(scores, axis = 0)
# and calculate accuracy (fraction of predictions that are correct)
np.mean(Yte_predict == Yte)
# returns 0.1555

이 방법으로 얻은 최선의 $W$의 정확도 15.5%이다. 완전 무작위로 guessing classes하는 것이 10%의 정확도를 보이므로, 무식한 방법 치고는 그리 나쁜지는 않다.

#### 핵심 아이디어: 반복적 향상

물론 이보다 더 좋은 방법들이 있다. <br /> 
여기서 핵심 아이디어는, 최선의 가중치들의 집합 $W$를 찾는 것이 매우 어렵거나 때로는 불가능한 문제(특히 복잡한 뉴럴 네트워크 전체를 구현할 경우)이지만, <br /> 어떤 특정 가중치들의 집합 $W$를 조금 개선시키는 문제는 상대적으로 쉽다는 점이다. <br /> 
다시 말해 무작위로 뽑은 $W$에서 출발하여 매번 조금씩 개선시키는 것을 반복하는 것이다.

> 핵심 전략은 무작위로 뽑은 파라미터로부터 시작해서 반복적으로 조금씩 개선시켜 손실을 낮추는 것이다.

#### Blindfolded hiker analogy

직관적인 이해를 돕기 위하여, 시야가 제한된 상태로 하이킹을 한다고 생각을 해보자. 목표는 the bottom에 도착하는 것이다. <br />
CIFAR-10의 예제에서, 봉우리는 30,730-차원이고, 따라서 $W$의 차원은 [10x3072]이다. 봉우리의 모든 점에서 특정 손실(지형의 height)를 얻을 수 있다.

#### 전략2: 무작위 국소 탐색

처음 떠오르는 전략은, 시작점에서 무작위로 방향을 정해서 발을 살짝 뻗어서 더듬어보고 그게 내리막길일 때만 한 발짝 내딛는 것이다. <br />
즉, 임의의 $W$에서 시작하여, 또다른 임의의 방향 $\delta W$으로 살짝 움직여본다. 만약에 움직여간 자리($W+\delta W$)에서의 손실값이 더 낮으면 <br />
거기로 움직이고 다시 탐색을 시작한다. 이 과정을 코드로 짜면 다음과 같다.

In [ ]:
W = np.random.randn(10, 3073) * 0.001 # 임의의 시작 파라미터를 랜덤하게 고른다.
bestloss = float("inf")
for i in xrange(1000):
  step_size = 0.0001
  Wtry = W + np.random.randn(10, 3073) * step_size
  loss = L(Xtr_cols, Ytr, Wtry)
  if loss < bestloss:
    W = Wtry
    bestloss = loss
  print 'iter %d loss is %f' % (i, bestloss)

이전과 같은 횟수(1000번)만큼 손실함수를 계산하고도, 이 방법을 테스트 데이터에 적용해보니, 분류 정확도가 21.4%로 나왔다. <br />
발전하기는 했지만, 여전히 비효율 적이다.

#### 전략3: 그라디언트 따라가기

이전 섹션에서는, 가중치 공간에서 가중치 벡터를 향상시키는(즉, 손실값을 더 낮추는) 방향을 찾는 시도를 해보았다. <br />
그런데 사실 좋은 방향을 찾기 위해 방향을 무작위로 탐색할 필요가 없다. (적어도 반지름이 0으로 수렴하는 아주 좁은 근방에서는) <br />
가장 가파르게 감소한다고 수학적으로 검증된 최선의 방향을 구할 수 있고, 이 방향을 따라 가중치 벡터를 움직이면 된다. <br />
이 방향은 손실함수의 그라디언트와 관계가있다. <br /> 
눈을 가리고 하산하는 것에 비유할 때, 발 밑의 지형을 더음어보고 가장 가파르다는 느낌을 주는 방향으로 내려가는 것에 비유할 수 있다.

1차원 함수의 경우, 어떤 점에서 움직일 때 기울기는 함수값의 순간 증가율을 나타낸다. 그라디언트는 이 기울기란 것을, 변수가 하나가 아니라 여러개의 경우로 일반화시킨 것이다. <br />
즉, 그라디언트는 입력 데이터 공간의 각 차원에 해당하는 기울기(미분)들의 벡터이다. 1차원 함수의 미분을 수식으로 쓰면 다음과 같다.

<img src="./Images/6.png" width=400 />

함수의 관점에서 숫자 하나가 아닌 벡터를 입력으로 받는 경우, 이때의 미분을 partial derivative라고 부르고, 그라디언트는 단순히 각 차원에서의 편미분들을 모아놓은 벡터이다.

## 그라디언트 계산

그라디언트 계산법은 크게 2가지가 있다. 느리고 근사값이지만 쉬운 방법(수치 그라디언트), 빠르고 정확하지만 미분이 필요하고 실수하기 쉬운 방법(해석적 그라디언트) <br />
여기서는 둘 다 다룰 것이다.

#### 유한한 차이를 이용하여 수치적으로 그라디언트 계산하기

위에 주어진 수식을 이용하여 그라디언트를 수치적으로 계산할 수 있다. <br />
여기에 임의의 함수 f와 이 함수에 입력값으로 넣을 벡터 x가 주어졌을 때, x에서 f의 그라디언트를 계산해주는 generic 함수가 있다.

In [ ]:
def eval_numerical_gradient(f, x):
  """ 
  a naive implementation of numerical gradient of f at x 
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """ 

  fx = f(x) # evaluate function value at original point
  grad = np.zeros(x.shape)
  h = 0.00001

  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    old_value = x[ix]
    x[ix] = old_value + h # increment by h
    fxh = f(x) # evalute f(x + h)
    x[ix] = old_value # restore to previous value (very important!)

    # compute the partial derivative
    grad[ix] = (fxh - fx) / h # the slope
    it.iternext() # step to next dimension

  return grad

이 코드는, 위에 주어진 그라디언트 식을 이용해하여 모든 차원을 하나씩 돌아가면서 그 방향으로 작은 변화 h를 주었을 때, <br />
손실 함수의 값이 얼마나 변하는지를 구해서, 그 방향의 편미분 값을 계산한다. 변수 grad에 전체 그라디언트 값이 최종적으로 저장된다. 

#### 실제로 고려할 사항

h가 0으로 수렴할 때의 극한값이 그라디언트의 수학적인 정의인데, 이 예시에서 나온 것처럼 ${ e }^{ -5 }$처럼 작은 값이면 충분하다. <br />
이상적으로, 수치적인 문제를 일으키지 않는 수준에서 가장 작은 값을 쓰면 된다. <br /> 
실제로 활용할 때, centered difference formula $ \frac { [f(x+h)-f(x-h)] }{ 2 }  $를 이용한 수치적 그라디언트의 결과가 더 좋은 경우가 많다.

위에서 주어진 함수를 이용하면, 어떤 함수의 어떤 값에서도 그라디언트를 계산할 수 있다. 가중치 공간의 임의의 점에서 CIFAR-10 손실함수의 그라디언트를 구해본다.

In [ ]:
# to use the generic code above we want a function that takes a single argument
# (the weights in our case) so we close over X_train and Y_train
def CIFAR10_loss_fun(W):
  return L(X_train, Y_train, W)

W = np.random.rand(10, 3073) * 0.001 # random weight vector
df = eval_numerical_gradient(CIFAR10_loss_fun, W) # get the gradient

그라디언트는 각 차원에 대하여 CIFAR-10의 손실함수의 기울기를 알려주는데, 그걸 이용해서 업데이트를 한다.

In [ ]:
loss_original = CIFAR10_loss_fun(W) # the original loss
print 'original loss: %f' % (loss_original, )

# lets see the effect of multiple step sizes
for step_size_log in [-10, -9, -8, -7, -6, -5,-4,-3,-2,-1]:
  step_size = 10 ** step_size_log
  W_new = W - step_size * df # new position in the weight space
  loss_new = CIFAR10_loss_fun(W_new)
  print 'for step size %f new loss: %f' % (step_size, loss_new)

# prints:
# original loss: 2.200718
# for step size 1.000000e-10 new loss: 2.200652
# for step size 1.000000e-09 new loss: 2.200057
# for step size 1.000000e-08 new loss: 2.194116
# for step size 1.000000e-07 new loss: 2.135493
# for step size 1.000000e-06 new loss: 1.647802
# for step size 1.000000e-05 new loss: 2.844355
# for step size 1.000000e-04 new loss: 25.558142
# for step size 1.000000e-03 new loss: 254.086573
# for step size 1.000000e-02 new loss: 2539.370888
# for step size 1.000000e-01 new loss: 25392.214036

#### Update in negative gradient direction

위 코드에서, W_new를 계산할 때 , 그라디언트 df의 반대방향으로 업데이트 하는 것에 주목하자. <br />
왜냐하면 우리가 원하는 것은 손실함수의 증가가 아니라 감소이기 때문이다.

#### 스텝의 크기가 미치는 영향

그라디언트에서 알 수 있는 것은 함수값이 가장 빠르게 증가하는 방향이고, 그 방향으로 대체로 얼마만큼을 가야하는지를 알려주지는 않는다. <br />
강의 뒤에서 다루게 되겠지만, 얼마만큼 가야하는지를 의미하는 스텝 크기(혹은 학습 속도)는 뉴럴 네트워크를 학습시키는데 있어서, 가장 중요한 하이퍼 파라미터이다. <br />
눈 가리고 하산을 하는 비유에서, 발 밑으로 어느 방향이 가장 가파른지 느끼지만, 얼마나 발을 뻗어야 할지는 불확실하다. <br /> 
발을 조심히 휘져으면서, 꾸준하지만 매우 조금씩 밖에 못 내려갈 것이다. (이는 아주 작은 스텝 크기에 비견된다.) <br />
반대로, 욕심껏 빨리 내려가려고 크고 과감하게 발을 내딛을 수도 있는데, 항상 뜻대로 되지는 않을지 모르다. <br /> 
위의 제시된 코드에서와 같이 어느 수준 이상의 큰 스텝의 크기는 오히려 손실 값을 증가시킨다.
___

<img src="./Images/7.png" width=400 />

작은 변화값이 주는 영향을 시각적으로 보여주는 그림 <br />
특정 지점 $W$에서 시작해서 그라디언트(혹은 여기에 -1을 곱한 값)를 계산한다. <br />
이 그라디언트에 -1을 곱한 방향, 즉 흰색 화살표 방향이 손실함수가 가장 빠르게 감소하는 방향이다. <br />
그 방향으로 조금 가는 것은 일관되지만 느리게 최적화를 진행시킨다. 반면에, 그 방향으로 너무 많이 가면, 더 많이 prgress되지만 위험성도 더 크다. <br />
큰 스텝 사이즈에서는, 결국에는 최소값을 지나쳐서 손실값이 더 커지는 지점까지 가게될 수도 있다. <br />
스텝 크기(학습 속도)는 가장 중요한 하이퍼 파라미터이기 때문에 매우 조심스럽게 결정되어야 한다.
___

#### 효율성의 문제

수치적 그라디언트의 복잡도는 파라미터의 수에 따라 선형적으로 늘어난다. <br />
위 예시에서 총 30,730의 파라미터가 있으므로 30,730번 손실 함수 값을 계산해서 그라디언트를 구해봐야 딱 한 번 업데이트 할 수 있다. <br />
요즘 쓰이는 뉴럴 네트워크들은 수천만개의 파라미터도 우스운데, 이런 경우 이 문제는 매우 심각해진다. 명백히 이 전략은 scalable 하지 않고, 더 좋은 전략이 있다.

## 미적분을 이용하여 해석적으로 그라디언트를 계산하기

수치적 그라디언트는 유한 차이를 이용해서 매우 단순하다. 하지만 근사값이라는 점과 <br /> 
(그라디언트의 진짜 정의는 "h"가 0으로 수렴할 때의 극한값인데, 여기서는 그냥 작은 "h"값을 쓰기 때문에) 계산이 비효율적이라는 것이 단점이다. <br />
두 번째 방법은 미적분을 이용해서 해석적으로 그라디언트를 구하는 것인데, 이는 (근사치가 아닌)정확한 수식을 이용하기 때문에 계산이 매우 빠르다. <br />
하지만 수치적으로 구한 그라디언트와는 다르게 구현하는데 실수를 하기쉽다. <br /> 
그래서 실제 활용에서는 해석적으로 구한 다음에 수치적으로 구한 것과 비교해보고, 틀린 경우 고치는 것이 일반적인 프로세스이다. <br />
이 과정을 그라디언트 체크라고 한다.

SVM 손실 함수를 예를 들어서 설명해 보자

<img src="./Images/8.png" width=400 />

가중치로 이 함수를 미분할 수 있다. 예를 들어 ${w}_{{y}_{i}}$로 미분하면 아래와 같이 된다.

<img src="./Images/9.png" width=400 />

여기서 $\mathbb{1}$은 정의함수라고 하는데, 쉽게 말해 괄호 안의 조건이 충족되면 1, 아니면 0인 값을 갖는다. <br /> 
이렇게 써놓으니 어려워 보이지만, 실제로 코딩으로 구현할 때는 원하는 차이(마진)을 못 만족시키는, 따라서 손실함수의 증가에 일조하는 클래스의 갯수를 세고, <br /> 
이 숫자를 데이터 벡터 $x_i$에 곱하면 이게 바로 그라디언트이다. 단, 이는 올바른 클래스에 해당하는 $W$의 행으로 미분했을 때의 그라디언트이다. <br /> 
$j\neq y_i$인 다른 행에 대한 그라디언트는 다음과 같다.

<img src="./Images/10.png" width=400 />

일단 그라디언트의 수식을 구하고나면, 그라디언트를 업데이트 시키는 것은 간단하다.

## 그라디언트 하강 (Gradient Descent)

이제 손실함수의 그라디언트를 계산할 줄 알게 됐는데, 그라디언트를 계속해서 계산하고 파라미터를 업데이트하는 것을 그레디언트 하강이라고 한다. <br />
Vanilla 버전은 아래와 같다.

In [ ]:
# Vanilla Gradient Descent

while True:
  weights_grad = evaluate_gradient(loss_fun, data, weights)
  weights += - step_size * weights_grad # perform parameter update

이 단순한 루프는 모든 뉴럴 네트워크의 핵심이다. <br /> 
다른 방법으로 최적화를 할 수 있는 방법(e.g. LBFGS)이 있긴 하지만, 현재로는 그라디언트 하강이 뉴럴 네트워크의 손실 함수를 최적화 하는 것으로서 가장 일반적이고 많이 쓰인다. <br />
이 강의에서, 이 루프에 이것 저것 세세하게 덧붙이기를(예를 들어, 업데이트 수식이 정확히 어떻게 되는지 등...)할 것이다. <br />
하지만 결과에 만족할 때까지 그라디언트를 따라서 움직인다는 기본적인 개념은 바뀌지 않는다.

#### 미니 배치 그라디언트 하강

Large-scale의 응용 사례에서, 학습데이터가 수백만개가 주어질 수 있다. <br /> 
따라서, 파라미터를 한 번 업데이트하려고 학습 데이터 전체를 계산에 사용하는 것은 낭비가 될 수 있다. <br />
이를 극복하기 위해서 흔하게 쓰이는 방법으로는 학습 데이터의 batches만 이용해서 그라디언트를 구하는 것이다. <br />
예를 들어, ConvNet을 쓸 때, 한 번에 120만개 중에 256개짜리 배치만을 이용해서 그라디언트를 구하고 파라미터를 업데이트 한다. 
다음 코드를 살펴보자

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
  data_batch = sample_training_data(data, 256) # sample 256 examples
  weights_grad = evaluate_gradient(loss_fun, data_batch, weights)
  weights += - step_size * weights_grad # perform parameter update

이 방법이 작 잘동하는 이유는 학습 데이터들 사이에 서로 상관관계가 있기 때문이다. <br /> 
이것에 대해 알아보기위해, ILSVRC의 120만개 이미지들이 사실은 1천개의 unique한 이미지들의 복제들 이라는 극단적인 경우를 생각해보자. <br />
(즉, 한 클래스 당 하나이고, 이 하나가 1,200번 복사된 것) <br />
그러면 명백한 것은, 이 1,200개의 이미지에서 그라디언트 값은 다 똑같다는 점이다. <br />
120만개 전체에 대하여 손실 값의 평균을 구하는 것은, 1000개로 이루어진 작은 subset에 대하여 손실 값의 평균을 구하는 것과 같다. <br />
실제로는 당연히 중복된 데이터를 주지는 않겠지만, 미니 배치에서만 계산하는 그라디언트는 모든 데이터를 써서 구하는 것의 근사값으로 괜찮게 쓰일 수 있다. <br />
따라서, 미니 배치 그라디언트를 계산해서 더 자주 파라미터를 업데이트하면 실제로 더 빠른 수렴을 하게 된다.

이 방법의 극단적인 형태는 미니 배치가 데이터 오직 한 개로 이루어졌을 때이다. 이는 확률 그라디언트 하강이라고 불린다. (Stochastic Gradient Descent) <br />
이건 상대적으로 보편적이지 않은데, 그 이유는 프로그램을 짤 때 vecterized된 코드를 최적화 하기 때문에, <br /> 
한 예제를 100번 계산하는 것보다 100개의 예제에서 1번 계산하는 것이 더 빠르기 때문이다. <br />
SGD는 엄밀한 의미에서는 예제 하나짜리 미니 배치에 대하여 그라디언트를 계산하는 것이지만, 많은 사람들이 그냥 MGD("Minibatch Gradient Descent")를 의미하면서 SGD("Stochastic Gradient Descent)라고 부르기도 한다. <br />
미니 배치의 크기도 하이퍼 파라미터이지만,이것을 교차 검증하는 일은 흔치 않다. <br />
이건 대체로 컴퓨터 메모리 크기의 한계에 따라 결정된다,즉 몇 몇 특정값(예를 들어, 32, 64, 128 같은 것)을 이용한다. <br />
2의 제곱수를 이용하는 이유는 많은 벡터 계산이 2의 제곱수가 입력될 때 더 빠르기 때문이다.

## Summary
___

<img src="./Images/11.png" width=500 />

정보 흐름 요약 <br />
(x, y)라는 고정된 데이터 쌍이 주어져 있다. 처음에는 무작위로 뽑은 가중치들로 시작해서 바꿔나간다. <br />
왼쪽에서 오른쪽으로 가면서, 스코어 함수가 각 클래스의 점수를 계산하고 그 값이 f벡터에 저장된다. 손실 함수는 두 가지 요소를 포함하고 있다. <br />
첫째, 데이터 손실은 스코어 함수 f와 라벨 y 사이의 호환성을 계산한다. 둘째, 정규화(regularization) 손실은 오직 가중치만의 함수이다. <br />
그라디언트 하강 과정에서, 가중치들로 미분한(혹은 데이터 값으로 추가로 미분한) 그라디언트를 계산하고, 이것을 이용해서 파라미터 값을 없데이트 한다.
___

In this section,

- We developed the intuition of the loss function as a high-dimensional optimization landscape in which we are trying to reach the bottom. The working analogy we developed was that of a blindfolded hiker who wishes to reach the bottom. In particular, we saw that the SVM cost function is piece-wise linear and bowl-shaped.
- We motivated the idea of optimizing the loss function with iterative refinement, where we start with a random set of weights and refine them step by step until the loss is minimized.
- We saw that the gradient of a function gives the steepest ascent direction and we discussed a simple but inefficient way of computing it numerically using the finite difference approximation (the finite difference being the value of h used in computing the numerical gradient).
- We saw that the parameter update requires a tricky setting of the step size (or the learning rate) that must be set just right: if it is too low the progress is steady but slow. If it is too high the progress can be faster, but more risky. We will explore this tradeoff in much more detail in future sections.
- We discussed the tradeoffs between computing the numerical and analytic gradient. The numerical gradient is simple but it is approximate and expensive to compute. The analytic gradient is exact, fast to compute but more error-prone since it requires the derivation of the gradient with math. Hence, in practice we always use the analytic gradient and then perform a gradient check, in which its implementation is compared to the numerical gradient.
- We introduced the Gradient Descent algorithm which iteratively computes the gradient and performs a parameter update in loop.

#### Coming-up

The core takeaway from this section is that the ability to compute the gradient of a loss function with respect to its weights (and have some intuitive understanding of it) is the most important skill needed to design, train and understand neural networks. In the next section we will develop proficiency in computing the gradient analytically using the chain rule, otherwise also referred to as backpropagation. This will allow us to efficiently optimize relatively arbitrary loss functions that express all kinds of Neural Networks, including Convolutional Neural Networks.